In [1]:
import warnings
warnings.simplefilter('ignore')

In [7]:
%matplotlib inline
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
# importar el arbol de desicion 
from sklearn.tree import DecisionTreeRegressor
# visualizar el arbol de desiciones
import graphviz
# exportar grafo a graphviz
from sklearn.tree import export_graphviz

# Ensemble
ensemble = utilizan submodelos y hacen la prediccions

In [8]:
model = DecisionTreeRegressor(max_depth=2)

x = pd.read_csv('x_opening.csv')
y = x['worldwide_gross']
x = x.drop('worldwide_gross', axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 1)
model.fit(x_train, y_train)

treedot = export_graphviz(model, out_file=None, feature_names=x.columns)
treedot

'digraph Tree {\nnode [shape=box] ;\n0 [label="opening_gross <= 41613376.0\\nmse = 4.491994363696598e+16\\nsamples = 1665\\nvalue = 141540319.054"] ;\n1 [label="opening_gross <= 22074047.0\\nmse = 1.3333822193127844e+16\\nsamples = 1506\\nvalue = 92999937.199"] ;\n0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;\n2 [label="mse = 4923666241203546.0\\nsamples = 1257\\nvalue = 64781848.271"] ;\n1 -> 2 ;\n3 [label="mse = 3.147813101998775e+16\\nsamples = 249\\nvalue = 235450289.735"] ;\n1 -> 3 ;\n4 [label="opening_gross <= 70351576.0\\nmse = 1.103981187159358e+17\\nsamples = 159\\nvalue = 601300162.289"] ;\n0 -> 4 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;\n5 [label="mse = 4.067538845924509e+16\\nsamples = 92\\nvalue = 440868287.554"] ;\n4 -> 5 ;\n6 [label="mse = 1.2226485798747571e+17\\nsamples = 67\\nvalue = 821594676.851"] ;\n4 -> 6 ;\n}'

In [10]:
graphviz.Source(treedot)

ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH